In [ ]:
#impor packages
import pandas as pd
import numpy as np
import itertools
import os
import pickle
import json

In [ ]:
input_path = "/home/jupyter/datasets/raw/"

In [ ]:
#read LABEVENTS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'LABEVENTS.csv', dtype = {"SUBJECT_ID":"object",
                                                                        "HADM_ID":"object",
                                                                        "ITEMID":"object",
                                                                        "VALUE":"object",
                                                                        "VALUENUM":"object",
                                                                        "VALUEUOM":"object",
                                                                        "FLAG":"object",
                                                                        "CHARTTIME":"object"})
                 

In [ ]:
# read D_LABITEMS.csv file
df_D_ITEMS = pd.read_csv(filepath_or_buffer=input_path+'D_LABITEMS.csv', dtype = {"ITEMID":"object"})

In [ ]:
#Dropping Row_ID 
df = df.drop(labels='ROW_ID',axis = 1)

In [ ]:
# Print datatype and non-null data points for each column
df.info(verbose=True,null_counts=True)

In [ ]:
df_D_ITEMS.info()

In [ ]:
#check contents of D_ITEMS
df_D_ITEMS

In [ ]:
#keep columns ITEMID and LABEL
df_D_ITEMS = df_D_ITEMS[['ITEMID','LABEL','FLUID','CATEGORY']]

In [ ]:
df_D_ITEMS.head()

In [ ]:
df = pd.merge(left=df,right=df_D_ITEMS,how = 'left',on = 'ITEMID')

In [ ]:
# check the merged dataframe
df.head(50)

In [ ]:
df.info()

In [ ]:
df['VALUE_NUM_UOM'] = df['ITEMID']+'-'+df['VALUENUM']+'-'+df['VALUEUOM']

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['VALUE','VALUENUM','VALUEUOM','ITEMID'])

In [ ]:
df.info()

In [ ]:
df = df.fillna('NaN')

In [ ]:
df.head()

In [ ]:
df['labevents'] = df['LABEL']+' '+df['FLUID']+' '+df['CATEGORY']+' '+df['VALUE_NUM_UOM']+' '+df['FLAG']

In [ ]:
df.head(10)

In [ ]:
df['labevents'] = df['labevents'].map(lambda x: x.split())

In [ ]:
df = df.drop(labels=['FLAG','LABEL','FLUID','CATEGORY','VALUE_NUM_UOM'], axis = "columns")

In [ ]:
# remove 'NaN' form all events
df['labevents'] = df['labevents'].map(lambda x: list(filter(lambda a: a!='NaN',x)))

In [ ]:
df.head(10)

In [ ]:
# create output path
mypath_output = "/home/jupyter/datasets/labevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [ ]:
# export the dataframe to JSON format
df.to_json(mypath_output+"labevents.json",orient = 'records') 